In [11]:
# preliminaries

from collections import *
from dataframe import *

import os

import nltk

In [36]:
# base
class TSBase(object):
    def build_sentence_db(self, document_set):
        """
        document_set is set or list of (review_id, review_text) tuple.
        """
        # internal document mapping, implicitly index is the internal id, same for ism (sentences)
        idm = [document for document in document_set]
        ism = [(doc_id, sentence) for doc_id, doc in enumerate(idm) for sentence in nltk.sent_tokenize(doc[1]) ]

        self.world_txt = " ".join([text[1] for text in idm])
        self.world_txt = " ".join(self.world_txt.split())
    
    def world(self):
        return self.world_txt

# test for base
if not os.path.exists('dataset'):
    print('fak, fix it')
else:
    import codecs
    
    DOC_PREFIX = 'dataset/text/documents'
    txts = os.listdir(DOC_PREFIX)[:6]
    counter = 0
    docs = deque()
    for t in txts:
        with codecs.open(os.path.join(DOC_PREFIX,t),'r','utf-8') as f:
            raw = f.read()
            doc_id = os.path.splitext(os.path.basename(f.name))[0]
            print(doc_id)
            docs.append((doc_id,raw))
            counter+=1
    
    tsbase = TSBase()
    tsbase.build_sentence_db(docs)
    print(tsbase.world())


12345
test test '\,


In [ ]:

# centrality of sentence := \sum qualified(words); where qualified(w) = if w in pseudo_document ? 1 : 0